## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
#### <center>ФБ-25 Савченко Єлизавета</center>

In [49]:
!pip install pandas --break-system-packages


Defaulting to user installation because normal site-packages is not writeable


Імпортуємо бібліотеки

In [50]:
import datetime
import requests
import urllib.request
import pandas as pd
import os

Скачуємо індекси для всіх областей. Додаємо в назву дату та час скачування

In [36]:
if os.path.exists('vhi'):
    os.rename('vhi', 'vhi_backup_' + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
os.mkdir('vhi')

for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Download complete")
        break

    date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    vhi_url = urllib.request.urlopen(url)
    file_name = f'vhi/vhi_{ids}_{date_now}.csv'
    out = open(file_name, 'wb')
    out.write(vhi_url.read())
    out.close()
    print(f"VHI from id {ids} was downloaded at {date_now}")

VHI from id 1 was downloaded at 2024-05-04_09-27-01
VHI from id 2 was downloaded at 2024-05-04_09-27-03
VHI from id 3 was downloaded at 2024-05-04_09-27-05
VHI from id 4 was downloaded at 2024-05-04_09-27-07
VHI from id 5 was downloaded at 2024-05-04_09-27-08
VHI from id 6 was downloaded at 2024-05-04_09-27-10
VHI from id 7 was downloaded at 2024-05-04_09-27-12
VHI from id 8 was downloaded at 2024-05-04_09-27-14
VHI from id 9 was downloaded at 2024-05-04_09-27-16
VHI from id 10 was downloaded at 2024-05-04_09-27-18
VHI from id 11 was downloaded at 2024-05-04_09-27-21
VHI from id 12 was downloaded at 2024-05-04_09-27-22
VHI from id 13 was downloaded at 2024-05-04_09-27-24
VHI from id 14 was downloaded at 2024-05-04_09-27-26
VHI from id 15 was downloaded at 2024-05-04_09-27-28
VHI from id 16 was downloaded at 2024-05-04_09-27-30
VHI from id 17 was downloaded at 2024-05-04_09-27-32
VHI from id 18 was downloaded at 2024-05-04_09-27-33
VHI from id 19 was downloaded at 2024-05-04_09-27-35
VH

Даємо імена стовбцям та зчитуємо файли у фрейм

In [58]:
folder = 'vhi'
files = os.listdir(folder)
df_all = pd.DataFrame()

for file_name in files:
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    if file_name == 'df_all.csv':
        continue
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)
    df['ProvinceId'] = int(file_name.split("_")[1])
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df = df.drop(columns=['empty'])
    df_all = pd.concat([df_all, df]).drop_duplicates().reset_index(drop=True)

print("Дані до очищення:")
print(df_all, '\n')

df_with_question_mark = df_all[df_all.apply(lambda x: x.astype(str).str.contains('\?', na=False)).any(axis=1)]
print("Дані, що містять знак питання:")
print(df_with_question_mark, '\n')
df_all = df_all.dropna(axis=0, how='any')
print(df_all, '\n')
print("Дані після очищення:")
df_cleaned = df_all.drop(df_with_question_mark.index)
print(df_cleaned)

Дані до очищення:
              Year  Week    SMN     SMT    VCI    TCI    VHI  ProvinceId
0             1982   2.0  0.092  264.40  65.82  31.42  48.62          27
1             1982   3.0  0.097  265.74  70.62  27.11  48.86          27
2             1982   4.0  0.101  267.73  71.82  20.73  46.28          27
3             1982   5.0  0.102  268.87  65.88  21.34  43.61          27
4             1982   6.0  0.103  269.80  58.54  24.88  41.71          27
...            ...   ...    ...     ...    ...    ...    ...         ...
58072         2024  14.0  0.190  290.40  65.07  12.06  38.58          13
58073         2024  15.0  0.209  290.95  62.82  20.96  41.90          13
58074         2024  16.0  0.224  290.69  58.30  38.34  48.34          13
58075         2024  17.0  0.233  291.06  51.65  56.57  54.14          13
58076  </pre></tt>   NaN    NaN     NaN    NaN    NaN    NaN          13

[58077 rows x 8 columns] 

Дані, що містять знак питання:
Empty DataFrame
Columns: [Year, Week, SMN, SMT,

Змінюємо індекси областей

In [38]:
ProvinceId_replacements = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["ProvinceId"] = df_all["ProvinceId"].replace(ProvinceId_replacements)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  ProvinceId
0      1982   2.0  0.092  264.40  65.82  31.42  48.62           5
1      1982   3.0  0.097  265.74  70.62  27.11  48.86           5
2      1982   4.0  0.101  267.73  71.82  20.73  46.28           5
3      1982   5.0  0.102  268.87  65.88  21.34  43.61           5
4      1982   6.0  0.103  269.80  58.54  24.88  41.71           5
...     ...   ...    ...     ...    ...    ...    ...         ...
58071  2024  13.0  0.173  289.05  65.07   8.82  36.95          10
58072  2024  14.0  0.190  290.40  65.07  12.06  38.58          10
58073  2024  15.0  0.209  290.95  62.82  20.96  41.90          10
58074  2024  16.0  0.224  290.69  58.30  38.34  48.34          10
58075  2024  17.0  0.233  291.06  51.65  56.57  54.14          10

[58050 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [39]:
def vhi(ProvinceId, year):
    filtered_df = df_all[(df_all["ProvinceId"] == ProvinceId) & (df_all["Year"] == year)]
    return filtered_df['VHI']

def vhi_min(ProvinceId, year):
    filtered_df = df_all[(df_all["ProvinceId"] == ProvinceId) & (df_all["Year"] == year)]
    return filtered_df['VHI'].min()

def vhi_max(ProvinceId, year):
    filtered_df = df_all[(df_all["ProvinceId"] == ProvinceId) & (df_all["Year"] == year)]
    return filtered_df['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)


In [40]:
vhi(6, 2005)

46317    47.42
46318    51.20
46319    53.68
46320    54.35
46321    54.66
46322    56.10
46323    56.81
46324    56.06
46325    56.27
46326    56.51
46327    56.42
46328    56.02
46329    54.84
46330    52.12
46331    49.52
46332    48.44
46333    53.62
46334    52.32
46335    52.21
46336    51.77
46337    53.50
46338    54.49
46339    54.25
46340    55.50
46341    57.29
46342    57.93
46343    58.44
46344    65.13
46345    59.35
46346    61.40
46347    63.51
46348    64.39
46349    66.13
46350    65.04
46351    60.86
46352    57.68
46353    56.59
46354    54.98
46355    53.23
46356    52.48
46357    55.20
46358    57.22
46359    57.66
46360    58.95
46361    56.50
46362    54.77
46363    54.48
46364    53.22
46365    50.36
46366    47.41
46367    45.59
Name: VHI, dtype: float64

In [41]:
vhi_min(6, 2005)


45.59

In [42]:
vhi_max(6, 2005)

66.13

Ряд VHI за вказаний діапазон років для вказаних областей

In [43]:
def vhi_range(year_min, year_max, ProvinceIds):
    if not isinstance(ProvinceIds, list) or not ProvinceIds:
        print('Empty or not a list')
        return
    filtered_df = df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['ProvinceId'].isin(ProvinceIds))]
    return filtered_df[['Year', 'VHI', 'ProvinceId']]


In [44]:
vhi_range(2000, 2004, [5, 2, 3])

,Year,VHI,ProvinceId
905,2000,27.46,5
906,2000,30.88,5
907,2000,33.00,5
908,2000,36.72,5
909,2000,39.12,5
...,...,...,...
22651,2004,55.14,2
22652,2004,52.90,2
22653,2004,50.57,2
22654,2004,50.45,2


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [45]:
def extreme_droughts(percent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['ProvinceId'].nunique()
    result = grouped[grouped > (25 * percent / 100)].reset_index()
    return result

In [60]:
extreme_droughts(1)

,Year,ProvinceId
0,1984,1
1,1986,2
2,1993,2
3,1994,1
4,1999,1
5,2000,6
6,2003,1
7,2007,5
8,2012,1


І так само для помірних посух

In [47]:
def moderate_droughts(percent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['ProvinceId'].nunique()
    count_areas = df_all['ProvinceId'].nunique()
    result = grouped[grouped > (count_areas * percent / 100)].reset_index()
    return result

In [48]:
moderate_droughts(10)

,Year,ProvinceId
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
